In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

sample = " if you want you"

char_set = list(set(sample))  # id -> char
char_dic = {w: i for i, w in enumerate(char_set)}

# settings
rnn_hidden_size = dic_size = len(char_dic)  # output size of each cell
batch_size = 1  # one sample data,one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)

sample_idx = [char_dic[c] for c in sample]  # char to index
x_data = tf.one_hot(sample_idx[:-1], dic_size)  # one hot
y_data = sample_idx[1:]

# split to input (char)length. This will decide unrolling size
x_split = tf.split(value=x_data, num_or_size_splits=[sequence_length])
outputs = x_split # No lstm

# softmax layer
softmax_w = tf.get_variable("softmax_w", [sequence_length, dic_size])
softmax_b = tf.get_variable("softmax_b", [dic_size])
outputs = outputs * softmax_w + softmax_b

outputs = tf.reshape(outputs, [batch_size, sequence_length, dic_size])
y_data = tf.reshape(y_data, [batch_size, sequence_length])
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(outputs, y_data, weights)
mean_loss = tf.reduce_mean(sequence_loss) / batch_size
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    _, l = sess.run([train_op, mean_loss])
    results = sess.run(outputs)
    for result in results:
        index = np.argmax(result, axis=1)
        print(''.join([char_set[t] for t in index]), l)

 fffffffwfftfff 2.36874
   f   fw   f   2.31018
   y   y    y   2.26116
   y   y    y   2.22185
   y   y    y   2.19219
   y   y    y   2.17118
y  y   y    y   2.1571
y  y   y    y   2.14809
y  y   y    y   2.14249
o  o   o    o   2.13895
o  o   o    o   2.13644
o  o   o    o   2.1342
o  o   o    o   2.1317
o  o   o    o   2.12867
o  o   o    o   2.1251
o  o   o    o   2.12111
o  o   o    o   2.11689
o  o   o    o   2.11266
o  o   o    o   2.10857
o  o   o    o   2.10474
o  o   o    o   2.10125
o  o   o    o   2.09818
y  y   y    y   2.09557
y  y   y    y   2.0934
y  y   y    y   2.09165
y  y   y    y   2.09025
y  y   y    y   2.08914
y  y   y    y   2.08826
y  y   y    y   2.08755
y  y   y    y   2.08695
y  y   y    y   2.0864
y  y   y    y   2.08584
y  y   y    y   2.08524
o  o   o    o   2.08459
o  o   o    o   2.08391
o  o   o    o   2.0832
o  o   o    o   2.0825
o  o   o    o   2.08182
o  o   o    o   2.0812
o  o   o    o   2.08066
o  o   o    o   2.0802
o  o   o    o   2.07983
o 

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

sample = " if you want y"

char_set = list(set(sample))  # id -> char
char_dic = {w: i for i, w in enumerate(char_set)}

# settings
rnn_hidden_size = dic_size = len(char_dic)  # output size of each cell
batch_size = 1  # one sample data,one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)

sample_idx = [char_dic[c] for c in sample]  # char to index
x_data = tf.one_hot(sample_idx[:-1], dic_size)  # one hot
y_data = sample_idx[1:]

# Make lstm with rnn_hidden_size (each unit input vector size)
lstm = rnn.BasicLSTMCell(rnn_hidden_size, state_is_tuple=True)
lstm = rnn.MultiRNNCell([lstm] * 1, state_is_tuple=True)

# split to input (char)length. This will decide unrolling size
x_data = tf.reshape(x_data, [-1, rnn_hidden_size])
x_split = tf.split(value=x_data, num_or_size_splits=batch_size)

# outputs: unrolling size x hidden size, state = hidden size
outputs, _states = rnn.static_rnn(lstm, x_split, dtype=tf.float32)

# (optional) softmax layer
with tf.name_scope('softmax_layer') as scope:
    softmax_w = tf.get_variable("softmax_w", [sequence_length, dic_size])
    softmax_b = tf.get_variable("softmax_b", [dic_size])
    outputs = outputs * softmax_w + softmax_b

outputs = tf.reshape(outputs, [batch_size, sequence_length, dic_size])
y_data = tf.reshape(y_data, [batch_size, sequence_length])
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(outputs, y_data, weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(x_split))

for i in range(1000):
    _, l = sess.run([train_op, mean_loss])
    results = sess.run(outputs)
    for result in results:
        index = np.argmax(result, axis=1)
        print(''.join([char_set[t] for t in index]), l)

[array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)]
ttttttttttttt 2.43632
ttttttttttttt 2.35814
ttttttttttttt 2.28055
tt ytt ttttty 2.19769
 t yt   tyt y 2.10744
 t yy   yyt y 2.01093
   yy   y t y 1.90957
   yy   y t y 1.80359
   yy   y t y 1.69286
i  yy     t y 1.5

if you want y 0.00158764
if you want y 0.00158233
if you want y 0.00157705
if you want y 0.00157178
if you want y 0.00156657
if you want y 0.00156136
if you want y 0.00155615
if you want y 0.00155103
if you want y 0.00154589
if you want y 0.00154077
if you want y 0.00153571
if you want y 0.00153068
if you want y 0.00152567
if you want y 0.00152069
if you want y 0.00151572
if you want y 0.00151076
if you want y 0.00150588
if you want y 0.00150096
if you want y 0.00149609
if you want y 0.00149125
if you want y 0.00148644
if you want y 0.00148167
if you want y 0.0014769
if you want y 0.00147213
if you want y 0.00146743
if you want y 0.00146276
if you want y 0.00145811
if you want y 0.00145343
if you want y 0.00144882
if you want y 0.00144422
if you want y 0.00143965
if you want y 0.0014351
if you want y 0.00143059
if you want y 0.00142605
if you want y 0.00142161
if you want y 0.0014171
if you want y 0.00141268
if you want y 0.00140826
if you want y 0.00140389
if you want y 0.00139952
if 

if you want y 0.00059261
if you want y 0.000591492
if you want y 0.000590355
if you want y 0.000589219
if you want y 0.000588119
if you want y 0.000586992
if you want y 0.000585865
if you want y 0.000584783
if you want y 0.000583656
if you want y 0.000582547
if you want y 0.000581484
if you want y 0.000580348
if you want y 0.000579276
if you want y 0.000578213
if you want y 0.000577085
if you want y 0.000576004
if you want y 0.000574895
if you want y 0.00057385
if you want y 0.00057276
if you want y 0.000571687
if you want y 0.000570634
if you want y 0.000569589
if you want y 0.000568489
if you want y 0.000567417
if you want y 0.00056639
if you want y 0.000565309
if you want y 0.000564301
if you want y 0.00056321
if you want y 0.000562165
if you want y 0.000561148
if you want y 0.000560094
if you want y 0.000559059
if you want y 0.000558051
if you want y 0.000556997
if you want y 0.00055597
if you want y 0.000554962
if you want y 0.000553945
if you want y 0.000552927
if you want y 0.00